In [1]:
import polars as pl
from statsmodels.formula import api as smf

Read datasets as downloaded from [kaggle](https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020).

In [2]:
results = pl.read_csv("data/raw/results.csv", ignore_errors=True)
status = pl.read_csv("data/raw/status.csv")
drivers = pl.read_csv("data/raw/drivers.csv")
races = pl.read_csv("data/raw/races.csv")
constructors = pl.read_csv("data/raw/constructors.csv")

Since our model will try to predict the finishing position, we introduce a dummy variable to account for retirements from the race that the driver had no fault in. 
These are mostly techinical failures.
But there are some categories which still need further investigation.
These are marked in the following list with `# TODO`.

In [3]:
not_self_responsible_dnf = [
    "Engine",
    "Gearbox",
    "Transmission",
    "Clutch",
    "Hydraulics",
    "Electrical",
    "Radiator",
    "Suspension",
    "Brakes",
    "Differential",
    "Overheating",
    "Mechanical",
    "Tyre",
    "Driver Seat",
    "Puncture",
    "Driveshaft",
    "Retired", # TODO
    "Fuel pressure",
    "Front wing",
    "Water pressure",
    "Refuelling",
    "Wheel",
    "Throttle",
    "Steering",
    "Technical",
    "Electronics",
    "Broken wing",
    "Heat shield fire",
    "Exhaust",
    "Oil leak",
    "Wheel rim",
    "Water leak",
    "Fuel pump",
    "Track rod",
    "Oil pressure",
    "Engine fire",
    "Engine misfire",
    "Tyre puncture",
    "Out of fuel",
    "Wheel nut",
    "Pneumatics",
    "Handling", # TODO
    "Rear wing",
    "Fire",
    "Wheel bearing",
    "Physical", # TODO
    "Fuel system",
    "Oil line",
    "Fuel rig",
    "Launch control", # TODO
    "Injured", # TODO
    "Fuel",
    "Power loss",
    "Vibrations",
    "Safety",
    "Drivetrain",
    "Ignition"
    "Injury", # TODO
    "Chassis",
    "Battery",
    "Stalled", # TODO
    "Halfshaft",
    "Crankshaft",
    "Safety concerns", # TODO
    "Not restarted", # TODO
    "Alternator",
    "Underweight",
    "Safety belt",
    "Oil pump",
    "Fuel leak",
    "Excluded", # TODO
    "Injection",
    "Distributor",
    "Driver unwell", # TODO
    "Turbo",
    "CV joint",
    "Water pump",
    "Fatal accident", # TODO
    "Spark plugs",
    "Fuel pipe",
    "Eye injury",
    "Oil pipe",
    "Axle",
    "Water pipe",
    "Magneto", # TODO
    "Supercharger",
    "Collision damage", # TODO
    "Power Unit",
    "ERS",
    "Brake duct",
    "Seat",
    "Damage", # TODO
    "Debris", # TODO
    "Illness", # TODO
    "Undertray",
    "Cooling system",
]

In [4]:
finishing_position = "positionOrder"
df = (
    results
    # remove drivers with multiple entries per race and use their best
    .sort(["raceId", "driverId", finishing_position])
    .unique(["raceId", "driverId"], keep="first")
    # remove drivers with too few races
    .with_columns(pl.col("points").n_unique().over("driverId").alias("lifetime_points_driver"))
    .filter(pl.col("lifetime_points_driver") > 0)
    # count last position
    .with_columns(pl.col(finishing_position).max().over("raceId").alias(f"{finishing_position}_last"))
    # join dnf reasons
    .join(status, on="statusId", how="left")
    .with_columns(
        score = 1 - ((pl.col(finishing_position) - 1) / (pl.col(f"{finishing_position}_last") - 1)),
        technical_dnf = pl.col("status").unique().is_in(not_self_responsible_dnf).all(),
    )
    # build more human readable names
    .join(drivers, on="driverId", how="left")
    .join(races, on="raceId", how="left")
    .with_columns(
        race=pl.col("year").cast(pl.Utf8) + " " + pl.col("name"),
        driver=pl.col("forename") + " " + pl.col("surname") + " ("+  pl.col("code") + ")"
    )
    # build car info
    .join(constructors, on="constructorId", how="left", suffix="_constructor")
    .with_columns(
        car=pl.col("year").cast(pl.Utf8) + " " + pl.col("name_constructor")
    )
    .select(["raceId", "driverId", "race", "driver", "car", "score", "technical_dnf"])
).to_pandas().set_index(["raceId", "driverId"])

In [5]:
df.head()

race                   driver  \
raceId driverId                                                        
1      2         2009 Australian Grand Prix      Nick Heidfeld (HEI)   
       3         2009 Australian Grand Prix       Nico Rosberg (ROS)   
       4         2009 Australian Grand Prix    Fernando Alonso (ALO)   
       5         2009 Australian Grand Prix  Heikki Kovalainen (KOV)   
       15        2009 Australian Grand Prix       Jarno Trulli (TRU)   

                             car     score  technical_dnf  
raceId driverId                                            
1      2         2009 BMW Sauber  0.526316          False  
       3           2009 Williams  0.736842          False  
       4            2009 Renault  0.789474          False  
       5            2009 McLaren  0.052632          False  
       15            2009 Toyota  0.894737          False

Here is an example of the dataset used for modeling.
Most interesting is the `score` variable, which describes a drivers finishing position.
A value of `score=0` means the driver finished last.
A value of `score=1` means the driver won the race.

The model will try to preidct the `score` based on all other columns (excluding the `raceId` and `driverId` index).

The idea being that the finishing position is a good dependent variable when evalutating talent - if analysed together with other mitigating factors.
Most importantly those include the `car` and eventual techinical failures of the car `techincal_dnf`.
This is inspired by the method established in [this paper](https://www.researchgate.net/publication/228310743)'s section 3.

By furthermore restricting the `score` to `[0, 1]` we account for the fact that finishing 5th in a race of 10 competitors is a different achievment than finishing 5th in a race of 20.

In [6]:
model = smf.glm("score ~ driver + car + technical_dnf - 1", data=df).fit()

The models parameters will be our indicator to figure out the greatest of all time - both drivers and cars.
We extract only significant results and then sort by the lower boundary of 95% confidence interval.
The idea is to get a conservative estimate for the minimum skill ceiling of the drivers (or perfomrance ceiling of the car respectively).

Since a modern grid contains 20 drivers, we will show the top 20 for an all-star grid.
For the cars, we are only concerned with a top 10.

In [7]:
res = model.summary2().tables[1]
siginificant = res["P>|z|"] <= 0.05
drivers = res.index.str.startswith("driver")
cars = res.index.str.startswith("car")

In [8]:
res.loc[siginificant & drivers].sort_values("[0.025", ascending=False).head(20)

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
driver[Jim Clark (\N)],0.544198,0.046204,11.778241,5.053673e-32,0.453640,0.634755
driver[Juan Fangio (\N)],0.552844,0.053950,10.247432,1.215291e-24,0.447105,0.658583
driver[Max Verstappen (VER)],0.523711,0.049263,10.630885,2.140724e-26,0.427157,0.620265
driver[Lando Norris (NOR)],0.526951,0.059726,8.822814,1.116188e-18,0.409891,0.644012
driver[Emerson Fittipaldi (\N)],0.494364,0.043521,11.359071,6.685175e-30,0.409063,0.579664
driver[Alain Prost (\N)],0.467053,0.034583,13.505249,1.456199e-41,0.399271,0.534835
driver[Fernando Alonso (ALO)],0.472955,0.038500,12.284564,1.096346e-34,0.397496,0.548413
driver[Lewis Hamilton (HAM)],0.471506,0.040341,11.688052,1.467131e-31,0.392439,0.550573
driver[Nico Rosberg (ROS)],0.474935,0.043024,11.038894,2.480685e-28,0.390610,0.559260
driver[Charles Leclerc (LEC)],0.495170,0.053356,9.280507,1.686745e-20,0.390594,0.599745


In [9]:
res.loc[siginificant & cars].sort_values("[0.025", ascending=False).head(10)

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
car[T.1988 McLaren],0.472889,0.055204,8.566248,1.069117e-17,0.364692,0.581087
car[T.2011 Red Bull],0.474837,0.056600,8.389345,4.888580e-17,0.363903,0.585771
car[T.2004 Ferrari],0.467116,0.053873,8.670674,4.295715e-18,0.361527,0.572706
car[T.1990 Benetton],0.470450,0.056055,8.392651,4.753025e-17,0.360584,0.580316
car[T.2017 Mercedes],0.469364,0.056270,8.341306,7.347448e-17,0.359077,0.579651
car[T.2019 Mercedes],0.466043,0.055599,8.382231,5.193358e-17,0.357071,0.575015
car[T.2007 Ferrari],0.456140,0.055935,8.154844,3.496316e-16,0.346510,0.565770
car[T.1979 Ferrari],0.471237,0.064034,7.359171,1.850553e-13,0.345733,0.596741
car[T.1981 Brabham],0.452296,0.057608,7.851312,4.117089e-15,0.339387,0.565205
car[T.1992 Williams],0.443153,0.053601,8.267585,1.367003e-16,0.338097,0.548210
